In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)

In [6]:
X_1.shape

(7978, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt1 = DecisionTreeClassifier()
dt1.fit(X_1, y_1)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude2Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude2Attack/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [    31    118    919    882      1     38     53 111842      6     81]
(31, 56) (31,)
Save 0 to baseline_Def1.npy
(118, 56) (118,)
Save 1 to baseline_Def2.npy
(919, 56) (919,)
Save 2 to baseline_Def3.npy
(882, 56) (882,)
Save 3 to baseline_Def4.npy
(1, 56) (1,)
Save 4 to baseline_Def5.npy
(38, 56) (38,)
Save 5 to baseline_Def6.npy
(53, 56) (53,)
Save 7 to baseline_Def8.npy
(111842, 56) (111842,)
Save 8 to baseline_Def9.npy
(6, 56) (6,)
Save 9 to baseline_Def10.npy
(81, 56) (81,)
Save 10 to baseline_Def11.npy
Execution Time: 4.111501 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [155496      7   1001    527     81 142665    152  41945     38      1]
(155496, 56) (155496,)
Save 0 to BIM_Def1.npy
(7, 56) (7,)
Save 1 to BIM_Def2.npy
(1001, 56) (1001,)
Save 2 to BIM_Def3.npy
(527, 56) (527,)
Save 3 to BIM_Def4.npy
(81, 56) (81,)
Save 4 to BIM_Def5.npy
(142665, 56) (142665,)
Save 5 to BIM_Def6.npy
(152, 56) (152,)
Save 7 to BIM_Def8.npy
(41945, 

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [     3     11    357    423      1      8      9 113153      2      4]
(3, 56) (3,)
Save 0 to baseline_Def1.npy
(11, 56) (11,)
Save 1 to baseline_Def2.npy
(357, 56) (357,)
Save 2 to baseline_Def3.npy
(423, 56) (423,)
Save 3 to baseline_Def4.npy
(1, 56) (1,)
Save 4 to baseline_Def5.npy
(8, 56) (8,)
Save 5 to baseline_Def6.npy
(9, 56) (9,)
Save 7 to baseline_Def8.npy
(113153, 56) (113153,)
Save 8 to baseline_Def9.npy
(2, 56) (2,)
Save 9 to baseline_Def10.npy
(4, 56) (4,)
Save 10 to baseline_Def11.npy
Execution Time: 1.054311 seconds
(341913,)
[0 2 3 4 5 8] [109614    104    127      2 178544  53522]
(109614, 56) (109614,)
Save 0 to BIM_Def1.npy
(104, 56) (104,)
Save 2 to BIM_Def3.npy
(127, 56) (127,)
Save 3 to BIM_Def4.npy
(2, 56) (2,)
Save 4 to BIM_Def5.npy
(178544, 56) (178544,)
Save 5 to BIM_Def6.npy
(53522, 56) (53522,)
Save 8 to BIM_Def9.npy
Execution Time: 2.279137 seconds
(341913,)
[0 2 5 7 8] [ 86106     79 201339      1  54388]
(86106, 

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude2Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude2Attack/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [   260    547   2247   2052    137    943   1254 105562    220    749]
(260, 56) (260,)
Save 0 to baseline_Def1.npy
(547, 56) (547,)
Save 1 to baseline_Def2.npy
(2247, 56) (2247,)
Save 2 to baseline_Def3.npy
(2052, 56) (2052,)
Save 3 to baseline_Def4.npy
(137, 56) (137,)
Save 4 to baseline_Def5.npy
(943, 56) (943,)
Save 5 to baseline_Def6.npy
(1254, 56) (1254,)
Save 7 to baseline_Def8.npy
(105562, 56) (105562,)
Save 8 to baseline_Def9.npy
(220, 56) (220,)
Save 9 to baseline_Def10.npy
(749, 56) (749,)
Save 10 to baseline_Def11.npy
Execution Time: 0.706340 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [150131     18   2661   4498   4882 115081     14   6825  57772      8
     23]
(150131, 56) (150131,)
Save 0 to BIM_Def1.npy
(18, 56) (18,)
Save 1 to BIM_Def2.npy
(2661, 56) (2661,)
Save 2 to BIM_Def3.npy
(4498, 56) (4498,)
Save 3 to BIM_Def4.npy
(4882, 56) (4882,)
Save 4 to BIM_Def5.npy
(115081, 56) (115081,)
Save 5 to BIM_Def6.npy
(14, 56

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Exclude2Attack/UNSW_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_Exclude2Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,31,118,919,882,1,38,0,53,111842,6,81,113971
1,XGB1,155496,7,1001,527,81,142665,0,152,41945,38,1,341913
2,XGB1,119488,6,749,534,17,179548,0,220,41310,40,1,341913
3,XGB1,155496,7,1001,527,81,142665,0,152,41945,38,1,341913
4,XGB1,73228,218,2382,79,6,247595,480,15129,2639,105,52,341913
5,XGB1,100558,213,1924,2079,35,136031,0,141,100797,28,107,341913
6,XGB1,1875,332,2901,2495,4,700,0,170,333193,26,217,341913
7,XGB1,77891,26,268,723,1,39,0,51,262834,16,64,341913
8,XGB1,181929,50,285,525,58,104498,0,116,54434,17,1,341913
9,XGB1,154567,6,1153,649,45,141793,0,142,43527,31,0,341913
